### Standard Lib Imports

In [1]:
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [2]:
from src import config
from src.helper import *

### Load Data
For this notebook, we will load in all the datasets and combine and compare them.

In [3]:
fnames = [f for f in os.listdir(config.DATA_PATH) if f.endswith('.csv')]

In [5]:
data = []

for fname in fnames:
    data.append(read_data(fname))

In [11]:
# Separate into individual dataframes 

availability = data[0]
prices = data[1]
product_details = data[2]
sales = data[3]
visits_on_pdp = data[4]

del data

In [12]:
# Correct column name in Product Details dataframe
product_details = product_details.rename({'product_id*31': 'product_id'}, axis=1)